<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Diplodatos-Kaggle-Competition" data-toc-modified-id="Diplodatos-Kaggle-Competition-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Diplodatos Kaggle Competition</a></span><ul class="toc-item"><li><span><a href="#Importamos-librerías-y-funciones" data-toc-modified-id="Importamos-librerías-y-funciones-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Importamos librerías y funciones</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Agregación-y-Limpieza" data-toc-modified-id="Agregación-y-Limpieza-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Agregación y Limpieza</a></span></li></ul></li><li><span><a href="#Sets:-Entrenamiento,-Validación-y-Test." data-toc-modified-id="Sets:-Entrenamiento,-Validación-y-Test.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Sets: Entrenamiento, Validación y Test.</a></span></li><li><span><a href="#XGBClasiffier" data-toc-modified-id="XGBClasiffier-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>XGBClasiffier</a></span><ul class="toc-item"><li><span><a href="#Entrenamiento" data-toc-modified-id="Entrenamiento-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Entrenamiento</a></span></li><li><span><a href="#Validación" data-toc-modified-id="Validación-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Validación</a></span></li><li><span><a href="#Resultados" data-toc-modified-id="Resultados-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Resultados</a></span></li></ul></li><li><span><a href="#GradientBoostingClassifier" data-toc-modified-id="GradientBoostingClassifier-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>GradientBoostingClassifier</a></span><ul class="toc-item"><li><span><a href="#Entrenamiento" data-toc-modified-id="Entrenamiento-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Entrenamiento</a></span></li><li><span><a href="#Validación" data-toc-modified-id="Validación-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Validación</a></span></li><li><span><a href="#Resultados" data-toc-modified-id="Resultados-1.5.3"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>Resultados</a></span></li></ul></li><li><span><a href="#Submit" data-toc-modified-id="Submit-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Submit</a></span></li></ul></li></ul></div>

# Diplodatos Kaggle Competition

## Importamos librerías y funciones

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from libs import *

In [3]:
def getDuplicateColumns(df): 
    duplicateColumnNames = set() 
    for x in range(df.shape[1]): 
        col = df.iloc[:, x] 
        for y in range(x + 1, df.shape[1]):  
            otherCol = df.iloc[:, y] 
            if col.equals(otherCol): 
                duplicateColumnNames.add(df.columns.values[y])  
    return list(duplicateColumnNames) 

## Dataset

In [4]:
df_train = pd.read_csv("./data/train.csv")
df_train['is_train_set'] = 1
df_test = pd.read_csv("./data/test.csv")
df_test['is_train_set'] = 0
df_train.TripType.replace({999: 0}, inplace=True) # Reemplazamos 999 por 0 (Para que no sea un valor extremos al resto)
df_train.TripType = df_train.TripType*0.01        # Dividimos por 100 para tener los datos entre 0 y 1

In [5]:
display(df_train)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,is_train_set
0,0.00,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0,1
1,0.08,9,Friday,1.070081e+09,1,IMPULSE MERCHANDISE,115.0,1
2,0.08,9,Friday,3.107000e+03,1,PRODUCE,103.0,1
3,0.08,9,Friday,4.011000e+03,1,PRODUCE,5501.0,1
4,0.08,10,Friday,6.414410e+09,1,DSD GROCERY,2008.0,1
...,...,...,...,...,...,...,...,...
453406,0.22,191344,Sunday,7.315096e+10,1,BEAUTY,3405.0,1
453407,0.22,191344,Sunday,6.505300e+10,1,WIRELESS,1712.0,1
453408,0.22,191344,Sunday,7.918131e+09,1,BEAUTY,3405.0,1
453409,0.08,191347,Sunday,4.190008e+09,1,DAIRY,1512.0,1


### Agregación y Limpieza

In [6]:
#Agrupamos por la Moda
y = df_train[['VisitNumber', 'Weekday', 'TripType']].groupby(['VisitNumber', 'Weekday']).agg(lambda x:x.value_counts().index[0])['TripType'].values

In [7]:
df = pd.concat([df_train.drop('TripType', axis=1), df_test])
df.reset_index(inplace=True, drop=True)

In [8]:
#Nos fijamos cuáles Departaments poseen Upc nulo
df[df.Upc.isnull()].groupby(['VisitNumber', 'Weekday', 'DepartmentDescription'], as_index=False).count().DepartmentDescription.unique()

array(['PHARMACY RX'], dtype=object)

Solo hay nulos en UPC (agrupados) en _PHARMACY RX_. Rellenaremos esos nulos con algun valor random extraído del historial de Upc para entradas de _PHARMACY RX_

In [9]:
values = df[(df.DepartmentDescription=='PHARMACY RX') & (~df.Upc.isnull())].Upc.values
df.Upc = df.Upc.apply(lambda x: np.random.choice(values) if (np.isnan(x)) else x)

In [10]:
## df.Upc            = df.Upc.fillna(-1)
## df.FinelineNumber = df.FinelineNumber.fillna(-1)

Ahora rellenamos FileNumber  y DepartmentDescription con un método parecido.
Asociamos a los _FinelineNumber_ nulos, el valor asociado al Filenumber existente correspondiente Upc de su entrada. (Igual para DepartmentDescription)

In [11]:
#FileNumber
valores = []
for i in np.where(df.FinelineNumber.isnull())[0]:
    valor = np.random.choice(df[(df.Upc == df.Upc.iloc[i]) & ~df.FinelineNumber.isnull()].FinelineNumber.values)
    valores.append([i,valor])
valores = np.asarray(valores)
fine = list(df.FinelineNumber)
for i in range(len(valores)):
    fine[(valores[i,0]).astype(int)] = valores[i,1]
df.FinelineNumber = fine

In [12]:
#DepartmentDescription
valores2 = []
for i in np.where(df.DepartmentDescription.isnull())[0]:
    valor = np.random.choice(df[(df.Upc == df.Upc.iloc[i]) & ~df.DepartmentDescription.isnull()].DepartmentDescription.values)
    valores2.append([i,valor])
valores2 = np.asarray(valores2)
Depa = list(df.DepartmentDescription)
for i in range(len(valores2)):
    Depa[(valores2[i,0]).astype(int)] = valores2[i,1]
df.DepartmentDescription = Depa

In [13]:
#Numeramos los días de la semana
df.Weekday.replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}, inplace=True)

Creamos columnas nuevas

In [14]:
RowxVis       = dict(df.groupby(['VisitNumber'])['Upc'].count())
df['RowxVis'] = df['VisitNumber'].apply(lambda x:RowxVis.get(x,0))     #Columnas por visita
RowxWee       = dict(df.groupby(['Weekday'])['Upc'].count())
df['RowxWee'] = df['Weekday'].apply(lambda x:RowxWee.get(x,0))         #Columnas por semana
UpcxVis       = dict(df.groupby(['VisitNumber'])['Upc'].nunique())
df['UpcxVis'] = df['VisitNumber'].apply(lambda x:UpcxVis.get(x,0))     #Upc únicos por visita
UpcxWee       = dict(df.groupby(['Weekday'])['Upc'].nunique())
df['UpcxWee'] = df['Weekday'].apply(lambda x:UpcxWee.get(x,0))         #Upc únicos por semana
df['ItemNum'] = df.apply(lambda x: str(x.Upc).zfill(11)[-6:-1],
                         axis=1).astype(float).astype(int)             #Número de Ítem
ItemxVis      = dict(df.groupby(['VisitNumber'])['ItemNum'].nunique())
df['ItxVis']  = df['VisitNumber'].apply(lambda x:ItemxVis.get(x,0))    #Ítems únicos por visita
ItemxWee      = dict(df.groupby(['Weekday'])['ItemNum'].nunique())
df['ItxWee']  = df['Weekday'].apply(lambda x:ItemxWee.get(x,0))        #Ítems únicos por semana
df['ManNum']  = df.apply(lambda x: str(x.Upc).zfill(11)[:6],
                         axis=1).astype(float).astype(int)             #Número de Manufact
ManxVis       = dict(df.groupby(['VisitNumber'])['ManNum'].nunique())
df['ManxVis'] = df['VisitNumber'].apply(lambda x:ManxVis.get(x,0))     #Manufacts únicos por visita
ManxWee       = dict(df.groupby(['Weekday'])['ManNum'].nunique())
df['ManxWee'] = df['Weekday'].apply(lambda x:ManxWee.get(x,0))         #Manufacts únicos por semana
ScanxVis      = dict(df.groupby(['VisitNumber'])['ScanCount'].sum())
df['ScaxVis'] = df['VisitNumber'].apply(lambda x:ScanxVis.get(x,0))    #Scans únicos por visita
ScanxWee       = dict(df.groupby(['Weekday'])['ScanCount'].sum())
df['ScaxWee'] = df['Weekday'].apply(lambda x:ScanxWee.get(x,0))        #Scans únicos por semana

In [15]:
#Hot encoding de los Depart (sin Nan)
df = BinarAll(df, ['DepartmentDescription'], merge=True)
# df.drop('DepartmentDescription_nan', axis=1, inplace=True)

In [16]:
display(df)

,VisitNumber,Weekday,Upc,ScanCount,FinelineNumber,is_train_set,RowxVis,RowxWee,UpcxVis,UpcxWee,ItemNum,ItxVis,ItxWee,ManNum,ManxVis,...,DepartmentDescription_PHARMACY RX,DepartmentDescription_PLAYERS AND ELECTRONICS,DepartmentDescription_PLUS AND MATERNITY,DepartmentDescription_PRE PACKED DELI,DepartmentDescription_PRODUCE,DepartmentDescription_SEAFOOD,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS
0,5,5,6.811315e+10,-1,1000.0,1,1,96247,1,38136,2929,1,9561,681131,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,5,1.070081e+09,1,115.0,1,3,96247,3,38136,727,3,9561,107008,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,9,5,3.107000e+03,1,103.0,1,3,96247,3,38136,3107,3,9561,3,3,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,9,5,4.011000e+03,1,5501.0,1,3,96247,3,38136,4011,3,9561,4,3,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,10,5,6.414410e+09,1,2008.0,1,3,96247,3,38136,235,3,9561,641441,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647049,191346,7,3.120033e+09,1,4639.0,0,17,133975,17,44141,3013,17,9704,312003,16,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647050,191346,7,3.700091e+09,1,8947.0,0,17,133975,17,44141,1229,17,9704,370009,16,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647051,191346,7,3.239000e+10,1,1118.0,0,17,133975,17,44141,1778,17,9704,323900,16,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647052,191346,7,7.874205e+09,1,1752.0,0,17,133975,17,44141,5336,17,9704,787420,16,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
#Agrupamos por suma
df2  = df.groupby(['VisitNumber', 'Weekday'], as_index=False).sum()

In [18]:
#Agrupamos algunas columnas por moda
df3 = df[['VisitNumber', 'Weekday', 'Upc', 'FinelineNumber', 'ItemNum', 'ManNum']
        ].groupby(['VisitNumber', 'Weekday'], as_index=False).agg(lambda x: x.mode()[0])

In [19]:
#Agrupamos algunas columnas por mediana
df4 = df[['VisitNumber', 'Weekday', 'ScanCount', 'RowxVis', 'RowxWee',
          'UpcxVis', 'UpcxWee', 'ItxVis', 'ItxWee', 'ManxVis', 'ManxWee',
         'ScaxVis', 'ScaxWee']
        ].groupby(['VisitNumber', 'Weekday'], as_index=False).median()

In [20]:
#Merge de todas las columnas
df = df2.merge(df3.merge(df4, on=['VisitNumber', 'Weekday']), on=['VisitNumber', 'Weekday'])

In [21]:
display(df)

,VisitNumber,Weekday,Upc_x,ScanCount_x,FinelineNumber_x,is_train_set,RowxVis_x,RowxWee_x,UpcxVis_x,UpcxWee_x,ItemNum_x,ItxVis_x,ItxWee_x,ManNum_x,ManxVis_x,...,Upc_y,FinelineNumber_y,ItemNum_y,ManNum_y,ScanCount_y,RowxVis_y,RowxWee_y,UpcxVis_y,UpcxWee_y,ItxVis_y,ItxWee_y,ManxVis_y,ManxWee_y,ScaxVis_y,ScaxWee_y
0,5,5,6.811315e+10,-1,1000.0,1,1,96247,1,38136,2929,1,9561,681131,1,...,6.811315e+10,1000.0,2929,681131,-1.0,1.0,96247.0,1.0,38136.0,1.0,9561.0,1.0,6180.0,-1.0,106945.0
1,7,5,6.794963e+10,2,13435.0,0,4,192494,4,76272,7079,4,19122,1346469,4,...,7.410811e+09,4504.0,1099,605388,1.0,2.0,96247.0,2.0,38136.0,2.0,9561.0,2.0,6180.0,2.0,106945.0
2,8,5,4.940370e+11,28,64284.0,0,529,2213681,483,877128,71236,460,219903,9990892,391,...,2.238400e+09,1017.0,0,200661,1.0,23.0,96247.0,21.0,38136.0,20.0,9561.0,17.0,6180.0,28.0,106945.0
3,9,5,1.070088e+09,3,5719.0,3,9,288741,9,114408,7845,9,28683,107015,9,...,3.107000e+03,103.0,727,3,1.0,3.0,96247.0,3.0,38136.0,3.0,9561.0,3.0,6180.0,3.0,106945.0
4,10,5,1.700927e+10,3,10073.0,3,9,288741,9,114408,5107,9,28683,1700926,9,...,2.800054e+09,115.0,235,280005,1.0,3.0,96247.0,3.0,38136.0,3.0,9561.0,3.0,6180.0,3.0,106945.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,7,4.516225e+11,9,33991.0,7,49,937825,49,308987,28170,49,67928,5263223,42,...,3.400000e+09,5225.0,246,885306,1.0,7.0,133975.0,7.0,44141.0,7.0,9704.0,6.0,6493.0,9.0,148150.0
95670,191344,7,1.614572e+11,5,15127.0,5,25,669875,25,220705,13497,25,48520,3707365,20,...,7.417002e+09,3405.0,1034,791813,1.0,5.0,133975.0,5.0,44141.0,5.0,9704.0,4.0,6493.0,5.0,148150.0
95671,191345,7,1.191780e+11,17,49902.0,0,169,1741675,169,573833,56504,169,126152,5770481,143,...,1.380310e+09,2764.0,201,370008,1.0,13.0,133975.0,13.0,44141.0,13.0,9704.0,11.0,6493.0,17.0,148150.0
95672,191346,7,9.870798e+10,17,72908.0,0,289,2277575,289,750397,48384,289,164968,6955697,272,...,4.072000e+03,4639.0,33,787420,1.0,17.0,133975.0,17.0,44141.0,17.0,9704.0,16.0,6493.0,17.0,148150.0


Creamos columnas nuevas

In [23]:
df['Weekend']  = ((df.Weekday==6) | (df.Weekday==7)).astype(int)   #Fin de semana
df['Devol']    = (df.ScanCount_y < 0).astype(int)                  #Devolución
df['Compra']   = (df.ScanCount_y != 0).astype(int)                 #Compra
df['Dev_Mean'] = (df.ScanCount_x < 0).astype(int)                  #Si en promedio fue devolución
df['Com_Mean'] = (df.ScanCount_x != 0).astype(int)                 #Si en promedio fue compra
df['FineCut']  = pd.qcut(df.FinelineNumber_x, q=20, labels=False)  #Repartimos en quartiles el filenumber
df             = BinarAll(df, 'FineCut', merge=True)               #Hot encoding quartiles de filenumber
df             = BinarAll(df, 'Weekday', merge=True)               #Hot encoding Día semanal

In [24]:
display(df)

,VisitNumber,Upc_x,ScanCount_x,FinelineNumber_x,is_train_set,RowxVis_x,RowxWee_x,UpcxVis_x,UpcxWee_x,ItemNum_x,ItxVis_x,ItxWee_x,ManNum_x,ManxVis_x,ManxWee_x,...,FineCut_12,FineCut_13,FineCut_14,FineCut_15,FineCut_16,FineCut_17,FineCut_18,FineCut_19,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Weekday_7
0,5,6.811315e+10,-1,1000.0,1,1,96247,1,38136,2929,1,9561,681131,1,6180,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,7,6.794963e+10,2,13435.0,0,4,192494,4,76272,7079,4,19122,1346469,4,12360,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,8,4.940370e+11,28,64284.0,0,529,2213681,483,877128,71236,460,219903,9990892,391,142140,...,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,9,1.070088e+09,3,5719.0,3,9,288741,9,114408,7845,9,28683,107015,9,18540,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,10,1.700927e+10,3,10073.0,3,9,288741,9,114408,5107,9,28683,1700926,9,18540,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,4.516225e+11,9,33991.0,7,49,937825,49,308987,28170,49,67928,5263223,42,45451,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
95670,191344,1.614572e+11,5,15127.0,5,25,669875,25,220705,13497,25,48520,3707365,20,32465,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
95671,191345,1.191780e+11,17,49902.0,0,169,1741675,169,573833,56504,169,126152,5770481,143,84409,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
95672,191346,9.870798e+10,17,72908.0,0,289,2277575,289,750397,48384,289,164968,6955697,272,110381,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [25]:
#Checkeamos columnas duplicadas
duplicateColNames = getDuplicateColumns(df) 
print('Las columnas duplicadas son:') 
for column in duplicateColNames : 
    print('Columna:', column)
# df.drop('Com_Mean', axis=1, inplace=True) #Eliminada manualmente

Las columnas duplicadas son:


## Sets: Entrenamiento, Validación y Test.

In [26]:
X_train = df[df.is_train_set != 0].drop('is_train_set', axis=1)
X_test  = df[df.is_train_set == 0].drop('is_train_set', axis=1)

In [27]:
SS_tr = StandardScaler() #Escalamos los datos
SS_tr.fit(X_train)
X_train_SS_tr = SS_tr.transform(X_train)
X_test_SS_tr  = SS_tr.transform(X_test)

In [28]:
random_state = 20 #Definimos semilla random

In [29]:
print('Ratio Test/Train:', len(X_test_SS_tr)/len(X_train_SS_tr))

Ratio Test/Train: 0.42735234003192646


In [30]:
x_train, x_valid, y_train, y_valid = train_test_split(
    X_train_SS_tr, y, test_size=0.18, random_state=random_state) #Separamos
print('Tamaño X_train:', x_train.shape)
print('Tamaño X_valid:', x_valid.shape)
print('Tamaño X_test :', X_test.shape)

Tamaño X_train: (54963, 130)
Tamaño X_valid: (12066, 130)
Tamaño X_test : (28645, 130)


## XGBClasiffier

### Entrenamiento

In [31]:
#XGBClasiffier, con HP tunneados (VARIAS pruebas realizadas)
xgb = XGBClassifier(random_state=random_state, learning_rate=0.02, n_estimators=600,
              objective='binary:logistic', subsample=0.8, min_child_weight=5,
              max_depth=5, gamma=1.5, colsample_bytree=0.8) 

In [32]:
xgb.fit(x_train, y_train, eval_set=[(x_valid,y_valid)], early_stopping_rounds=25)

[0]	validation_0-merror:0.41016
Will train until validation_0-merror hasn't improved in 25 rounds.
[1]	validation_0-merror:0.38422
[2]	validation_0-merror:0.37179
[3]	validation_0-merror:0.36756
[4]	validation_0-merror:0.36690
[5]	validation_0-merror:0.36259
[6]	validation_0-merror:0.36077
[7]	validation_0-merror:0.35654
[8]	validation_0-merror:0.35786
[9]	validation_0-merror:0.35621
[10]	validation_0-merror:0.35530
[11]	validation_0-merror:0.35571
[12]	validation_0-merror:0.35455
[13]	validation_0-merror:0.35372
[14]	validation_0-merror:0.35281
[15]	validation_0-merror:0.35016
[16]	validation_0-merror:0.34991
[17]	validation_0-merror:0.35049
[18]	validation_0-merror:0.35074
[19]	validation_0-merror:0.34983
[20]	validation_0-merror:0.34817
[21]	validation_0-merror:0.34842
[22]	validation_0-merror:0.34692
[23]	validation_0-merror:0.34651
[24]	validation_0-merror:0.34436
[25]	validation_0-merror:0.34535
[26]	validation_0-merror:0.34452
[27]	validation_0-merror:0.34469
[28]	validation_0-m

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=1.5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=5,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=600, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=20, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Validación

In [33]:
#Recordemos que habíamos dividido por 100...
print(accuracy_score((y_train*100).astype(int), (xgb.predict(x_train)*100).astype(int)))
print(accuracy_score((y_valid*100).astype(int), (xgb.predict(x_valid)*100).astype(int)))

0.8027582191656205
0.7296535720205536


### Resultados

In [34]:
yNew_xgb = xgb.predict(np.asarray(X_test_SS_tr))
print((yNew_xgb*100).astype(int))

[ 9 40 21 ...  8 39 39]


In [35]:
yy_xgb = (yNew_xgb*100).astype(int) #Recordemos que habíamos dividido por 100...

yy_xgb = pd.Series(yy_xgb)

yy_xgb.replace({0:999}, inplace=True) #Recordemos que habíamos reemplazado 999 por 0...

## GradientBoostingClassifier

### Entrenamiento

In [59]:
GradientBoostingClassifier?

Init signature:
GradientBoostingClassifier(
    *,
    loss='deviance',
    learning_rate=0.1,
    n_estimators=100,
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    init=None,
    random_state=None,
    max_features=None,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    presort='deprecated',
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0,
)
Docstring:     
Gradient Boosting for classification.

GB builds an additive model in a
forward stage-wise fashion; it allows for the optimization of
arbitrary differentiable loss functions. In each stage ``n_classes_``
regression trees are fit on the negative gradient of the
binomial or multinomial deviance loss function. Binary classification
is a special case where only a single regression tree is induced.

Read more in the :

In [60]:
#GradientBoostingClassifier, con HP tunneados (VARIAS pruebas realizadas)
gbc = GradientBoostingClassifier(random_state=random_state, verbose=1,
                                 learning_rate=0.02, max_depth=5,
                                 validation_fraction=0.18, n_iter_no_change=25)

In [61]:
# xgb = XGBClassifier(random_state=random_state, learning_rate=0.08, max_depth=None)
gbc.fit(x_train, (y_train*100).astype(int))

      Iter       Train Loss   Remaining Time 
         1           2.7795           74.96m
         2           2.6384           77.75m
         3           2.5258           78.03m
         4           2.4314           77.67m
         5           2.3485           77.15m
         6           2.2747           76.52m
         7           2.2086           75.84m
         8           2.1475           75.11m
         9           2.0919           74.38m
        10           2.0402           73.66m
        20           1.6673           65.72m
        30           1.4364           57.62m
        40           1.2754           49.43m
        50           1.1568           41.22m
        60           1.0647           32.98m
        70           0.9932           24.74m
        80           0.9347           16.50m
        90           0.8869            8.25m
       100           0.8456            0.00s


GradientBoostingClassifier(learning_rate=0.02, max_depth=5, n_iter_no_change=25,
                           random_state=20, validation_fraction=0.18,
                           verbose=1)

### Validación

In [62]:
#Recordemos que habíamos dividido por 100...
print(accuracy_score((y_train*100).astype(int), gbc.predict(x_train)))
print(accuracy_score((y_valid*100).astype(int), gbc.predict(x_valid)))

0.7480486872987282
0.6914470412729985


### Resultados

In [63]:
yNew_gbc = gbc.predict(np.asarray(X_test_SS_tr))
print(yNew_gbc)

[ 8 26 21 ...  8 38 39]


In [64]:
yy_gbc = yNew_gbc

yy_gbc = pd.Series(yy_gbc)

yy_gbc.replace({0:999}, inplace=True) #Recordemos que habíamos reemplazado 999 por 0...

## Submit

In [65]:
X_test  = df[df.is_train_set == 0].drop('is_train_set', axis=1)

In [66]:
submission_xgb = pd.DataFrame(list(zip(X_test.VisitNumber, yy_xgb)), columns=["VisitNumber", "TripType"])
submission_gbc = pd.DataFrame(list(zip(X_test.VisitNumber, yy_gbc)), columns=["VisitNumber", "TripType"])

In [67]:
submission_xgb.to_csv("./data/submission_xgb.csv", header=True, index=False)
submission_gbc.to_csv("./data/submission_gbc.csv", header=True, index=False)